#### Load and Inspect

In [1]:
import pandas as pd

# Load the datasets
df_creators = pd.read_csv("data/raw/dim_creators.csv")
df_posts = pd.read_csv("data/raw/fact_campaign_posts.csv")

# Quick check to ensure data types are correct
print("Creators Shape:", df_creators.shape)
print("Posts Shape:", df_posts.shape)

Creators Shape: (500, 6)
Posts Shape: (2500, 8)


#### Data Merging (The "Join")

In [2]:
# Perform a Left Join to keep all post data and attach creator details
df_merged = pd.merge(df_posts, df_creators, on='creator_id', how='left')

# Convert date column to datetime objects for time-series analysis later
df_merged['date'] = pd.to_datetime(df_merged['date'])

print("Data successfully integrated.")

Data successfully integrated.


#### Feature Engineering (The "Commercial" Metrics)

In [3]:
# 1. Total Engagement (The 'Surface' Metric)
df_merged['total_engagement'] = df_merged['likes'] + df_merged['comments'] + df_merged['saves']

# 2. Engagement Rate (The 'Quality' Metric)
df_merged['er'] = (df_merged['total_engagement'] / df_merged['follower_count']) * 100

# 3. Assign Reward Costs (The 'Commercial' Logic)
# Sample Product = $15, Full Size = $50, Exclusive Kit = $120
cost_map = {'Sample Product': 15, 'Full Size Set': 50, 'Exclusive Kit': 120}
df_merged['reward_cost'] = df_merged['reward_type'].map(cost_map)

# 4. Cost Per Engagement (CPE) - THE WINNING METRIC
# This tells the business: "How many dollars did we spend to get 1 like/save/comment?"
df_merged['cpe'] = df_merged['reward_cost'] / df_merged['total_engagement']

print("Features Engineered: Total Engagement, ER, and CPE calculated.")

Features Engineered: Total Engagement, ER, and CPE calculated.


#### Handling Outliers & Cleaning

In [4]:
# Check for any missing values created during the join
if df_merged.isnull().values.any():
    df_merged = df_merged.fillna(0)
    print("Missing values found and filled with 0.")

# Save the final "Analysis-Ready" file to the processed folder
df_merged.to_csv("data/processed/final_campaign_analysis.csv", index=False)

print("Phase 2 Complete! File saved to: data/processed/final_campaign_analysis.csv")
df_merged.head()

Phase 2 Complete! File saved to: data/processed/final_campaign_analysis.csv


,post_id,creator_id,date,reward_type,likes,comments,saves,impressions,name,market,category,follower_count,join_date,total_engagement,er,reward_cost,cpe
0,P-28CF52,CR-0362,2025-12-15,Sample Product,265,49,16,7546,Brandon Crawford,Germany,Makeup,6467,2024-04-19,330,5.102830,15,0.045455
1,P-AE32B7,CR-0307,2026-01-24,Full Size Set,80,16,64,3485,Melissa Kim,USA,Skincare,2934,2024-06-05,160,5.453306,50,0.312500
2,P-2F490E,CR-0116,2025-09-24,Exclusive Kit,114,22,91,4061,Janice Rasmussen,UK,Skincare,3617,2024-04-13,227,6.275919,120,0.528634
3,P-D5750E,CR-0213,2025-08-18,Sample Product,246,46,15,10889,Travis Chapman,USA,Haircare,7222,2024-07-09,307,4.250900,15,0.048860
4,P-F3EBA4,CR-0470,2026-01-17,Exclusive Kit,35,6,2,2753,Jordan Jones,USA,Personal Care,1636,2024-12-06,43,2.628362,120,2.790698
